## Updating processed mobility data with additional records

First load in libraries

In [ ]:
# installing the python packages using dbutils
dbutils.library.installPyPI("numpy",'1.19')
dbutils.library.installPyPI("descartes")
dbutils.library.installPyPI("fiona")
dbutils.library.installPyPI("shapely")
dbutils.library.installPyPI("pyproj")
dbutils.library.installPyPI("matplotlib")
dbutils.library.installPyPI("pyspark")
dbutils.library.installPyPI("geospark")

# importing the required python packages
import time
import datetime
from datetime import date

from geospark.register import upload_jars
from geospark.register import GeoSparkRegistrator

upload_jars() # necessary to load in GeoSpark libraries manually installed to the server

GeoSparkRegistrator.registerAll(spark)

In [ ]:
# import a more extensive list of pySpark and GeoSpark packages
import warnings
warnings.filterwarnings("ignore")

# General pyspark SQL stuff
import pyspark
from pyspark.sql.types import *
import pyspark.sql.functions as fs
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.functions import count
from pyspark.sql.functions import col, countDistinct
from pyspark.sql.functions import broadcast

# date time functions needed for filtering
from pyspark.sql.functions import from_utc_timestamp, from_unixtime, to_date, to_timestamp
from pyspark.sql.functions import *
from pyspark.sql import types as t

# delta tables necessary for pyspark manipulation of deltas
from delta.tables import *

import pylab as plt
from pyspark.sql.functions import lag
from pyspark.sql.window import Window
from pyspark.sql.functions import acos, cos, sin, lit, toRadians

# Geospark-py and SQL stuff
import geospark
from geospark.register import GeoSparkRegistrator
from geospark.utils import GeoSparkKryoRegistrator, KryoSerializer
from geospark.register import upload_jars
from geospark.core.formatMapper.shapefileParser import ShapefileReader
from geospark.core import SpatialRDD
from geospark.sql.types import GeometryType

In [ ]:
# these help optimize the operations of Spark
# Reference for partitions: https://stackoverflow.com/questions/35800795/number-of-partitions-in-rdd-and-performance-in-spark and https://hackernoon.com/managing-spark-partitions-with-coalesce-and-repartition-4050c57ad5c4
# Reference for memory / ResultSize limits: https://stackoverflow.com/questions/47996396/total-size-of-serialized-results-of-16-tasks-1048-5-mb-is-bigger-than-spark-dr

spark.conf.set("spark.sql.shuffle.partitions", 5000) # above 2000 it uses a different process. Setting to 500 so ( ( 56 / 8) * 120) / 5000) = 168 MB per partition
spark.conf.set("spark.network.timeout", 10000)
spark.conf.set("spark.driver.memory", 0)
spark.conf.set("spark.driver.maxResultSize", '16g')
spark.conf.set("spark.serializer", KryoSerializer.getName)
# spark/conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
spark.conf.set("spark.kryo.registrator", GeoSparkKryoRegistrator.getName)
spark.conf.set("geospark.global.index","true") # test
spark.conf.set("geospark.join.gridtype","kdbtree")

spark.conf.set("spark.sql.execution.arrow.enabled", "false")

In [ ]:
%scala
// import various GeoSpark libraries. 
// Full set of libraries can be tediously found here: https://github.com/DataSystemsLab/GeoSpark/tree/master/core/src/main/java/org/datasyslab/geospark

import com.vividsolutions.jts.geom.{Coordinate, Geometry, GeometryFactory}
import com.vividsolutions.jts.geom.Geometry

import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader
import org.datasyslab.geospark.spatialRDD.SpatialRDD
import org.datasyslab.geospark.enums.{GridType,IndexType}
import org.datasyslab.geospark.spatialPartitioning
import org.datasyslab.geospark.spatialOperator.{JoinQuery}
import org.datasyslab.geosparksql.utils.{Adapter, GeoSparkSQLRegistrator}
import org.datasyslab.geosparksql.UDT._

import org.apache.spark.sql.catalyst.util.{ArrayData, GenericArrayData}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.{rand, to_date, from_unixtime, unix_timestamp, _}
import org.apache.spark.sql.types._

import spark.sqlContext.implicits._

GeoSparkSQLRegistrator.registerAll(sqlContext)

#### Declare variables and load data

In [ ]:
provider = 'Veraset'
# provider = 'Unacast'

In [ ]:
today = date.today()

out_gtable = f'IN_{provider}_SJ'

filter_range = -30

In [ ]:
if provider == 'Unacast':
  out_delta_path = "/mnt/CUBEIQ/esapv/India/delta_unacast"
else:
  out_delta_path = "/mnt/CUBEIQ/esapv/India/delta_veraset"

#### Load all the provider's data and India admin geospatial data

In [ ]:
# converting the long and lat columns into Point(X, Y) column using geospark

points = spark.sql(f"SELECT device_id, \
                           timestamp, \
                           to_date(from_unixtime(timestamp,'yyyy-MM-dd')) as date, \
                           ST_Point(CAST(lon AS Decimal(24,20)),CAST(lat AS Decimal(24,20))) AS geometry \
                   FROM {provider}_1 \
                   WHERE {provider}_1.country = 'IN' AND timestamp > unix_timestamp(date_add(current_date(), {filter_range})) ") # loads within current date - filter range


In [ ]:
# Load in the geospatial administrative files directly from a global table
admin5 = sql("SELECT * FROM IN_adm5")

In [ ]:
# now register spark dataframes as tables
points.createOrReplaceTempView("pts_tbl")
admin5.createOrReplaceTempView("admin5_tbl")

#### Define spatial join query, using GeoSpark SQL

Note that this code is lazy, meaning we don't actually call this operation until the deltaTable operations below

In [ ]:
# spatial join for adm5
intersect_query_adm5 = """
  SELECT 
        s.L1_CODE as adm1_code, 
        s.L2_CODE as adm2_code, 
        s.L3_CODE as adm3_code, 
        s.L4_CODE as adm4_code,
        s.geo_id as adm5_code, 
        p.device_id, 
        p.timestamp, 
        p.date,
        p.geometry 
  FROM pts_tbl AS p, admin5_tbl AS s 
  WHERE ST_Intersects(p.geometry, s.geometry)
"""

spatial_join_result_final = spark.sql(intersect_query_adm5)

### Import existing delta data, run update on it.

In [ ]:
deltaTable = DeltaTable.forPath(spark, out_delta_path)

Only update where ID *and* timestamp don't match, to ensure new records for existing IDs are populated.

In [ ]:
# Spatial joining and inserting missing data into the delta table
# Only missing records are SJ'd and merged. We find missing records by searching for unique ID + timestamp combinations -- if they aren't matched, we insert.

deltaTable.alias("dt_master").merge(
    spatial_join_result_final.alias("updates"),
    "dt_master.device_id = updates.device_id AND dt_master.timestamp = updates.timestamp") \
  .whenNotMatchedInsertAll() \
  .execute()

Optimization keeps your tables nicely streamlined for later analysis. This saves you money + time.

In [ ]:
spark.sql(f"OPTIMIZE {out_gtable} where date >= (date_add(current_date(),{filter_range})) ZORDER by adm5_code")